In [1]:
import pandas as pd

In [2]:
Df = pd.read_csv("./databreach.csv")
Df["Date"] = pd.to_datetime(Df["Date"])
Df["data_classes"] = [ i.split(",") for i in Df["data_classes"] ]

In [3]:
Df

,Date,Org,category,breach_count(Million),data_classes
0,2017-10-01,Yahoo,web services,3000.0,[Security questions and answers]
1,2018-03-01,UIDAI,AADHAR,1100.0,"[Names, Aadhar number, Bank account numbers]"
2,2014-01-01,Yahoo,web services,500.0,"[Names, Email addresses, Phone numbers, Passwo..."
3,2013-06-01,MySpace,social media,360.0,"[Names, Usernames, Dates of birth]"
4,2018-06-01,Exactis,web services,340.0,"[Phone numbers, Physical addresses, Email addr..."
5,2018-05-01,Twitter,social media,330.0,[Passwords]
6,2013-10-01,Adobe,e commerce,152.0,"[Usernames, Email addresses, Passwords, Passwo..."
7,2018-02-01,MyFitnessPal,health,150.0,"[Email addresses, IP addresses, Usernames, Pas..."
8,2017-09-01,Equifax,web services,148.0,"[Names, Physical addresses, Phone numbers, Dat..."
9,2014-03-01,ebay,e commerce,145.0,"[Names, Email addresses, Physical addresses, P..."


In [5]:
pd.DataFrame(Df.Org.unique()).to_csv("./list_orgs.csv")